In [ ]:
# This is necessary to recognize the modules
import os
import sys
from decimal import Decimal
import warnings

warnings.filterwarnings("ignore")

root_path = os.path.abspath(os.path.join(os.getcwd(), '../..'))
sys.path.append(root_path)

In [ ]:
from core.backtesting import BacktestingEngine

backtesting = BacktestingEngine(root_path=root_path, load_cached_data=False)

In [ ]:
from hummingbot.strategy_v2.executors.position_executor.data_types import TrailingStop
from controllers.directional_trading.peinjo import PeinjoControllerConfig
import datetime
from decimal import Decimal

# Controller configuration
connector_name = "binance"
trading_pair = "BTC-USDT"
interval = "1m"
length = 200
inner_mult = 1.0
outer_mult = 2.415
source = "hlc3"
filter_type = "SuperSmoother"
total_amount_quote = 1000
max_executors_per_side = 3
take_profit = 0.05
stop_loss = 0.007
trailing_stop_activation_price = 0.015
trailing_stop_trailing_delta = 0.003
time_limit = 60 * 60 * 24 * 2
cooldown_time = 60 * 15

start = int(datetime.datetime(2024, 10, 1).timestamp())
end = int(datetime.datetime(2024, 10, 6).timestamp())

# Creating the instance of the configuration and the controller
config = PeinjoControllerConfig(
    connector_name=connector_name,
    trading_pair=trading_pair,
    interval=interval,
    length=length,
    inner_mult=inner_mult,
    outer_mult=outer_mult,
    source=source,
    filter_type=filter_type,
    total_amount_quote=Decimal(total_amount_quote),
    take_profit=Decimal(take_profit),
    stop_loss=Decimal(stop_loss),
    trailing_stop=TrailingStop(activation_price=Decimal(trailing_stop_activation_price), trailing_delta=Decimal(trailing_stop_trailing_delta)),
    time_limit=time_limit,
    max_executors_per_side=max_executors_per_side,
    cooldown_time=cooldown_time,
)

In [ ]:
# Running the backtesting this will output a backtesting result object that has built in methods to visualize the results

backtesting_result = await backtesting.run_backtesting(config, start, end, interval)

In [ ]:
# Let's see what is inside the backtesting results
print(backtesting_result.get_results_summary())
backtesting_result.get_backtesting_figure()

In [ ]:
# 2. The executors dataframe: this is the dataframe that contains the information of the orders that were executed
import pandas as pd

executors_df = backtesting_result.executors_df
executors_df.head()

### Backtesting Analysis

### Scatter of PNL per Trade
This bar chart illustrates the PNL for each individual trade. Positive PNLs are shown in green and negative PNLs in red, providing a clear view of profitable vs. unprofitable trades.


In [ ]:
import plotly.express as px

# Create a new column for profitability
executors_df['profitable'] = executors_df['net_pnl_quote'] > 0

# Create the scatter plot
fig = px.scatter(
    executors_df,
    x="timestamp",
    y='net_pnl_quote',
    title='PNL per Trade',
    color='profitable',
    color_discrete_map={True: 'green', False: 'red'},
    labels={'timestamp': 'Timestamp', 'net_pnl_quote': 'Net PNL (Quote)'},
    hover_data=['filled_amount_quote', 'side']
)

# Customize the layout
fig.update_layout(
    xaxis_title="Timestamp",
    yaxis_title="Net PNL (Quote)",
    legend_title="Profitable",
    font=dict(size=12, color="white"),
    showlegend=False,
    plot_bgcolor='rgba(0,0,0,0.8)',  # Dark background
    paper_bgcolor='rgba(0,0,0,0.8)',  # Dark background for the entire plot area
    xaxis=dict(gridcolor="gray"),
    yaxis=dict(gridcolor="gray")
)

# Add a horizontal line at y=0 to clearly separate profits and losses
fig.add_hline(y=0, line_dash="dash", line_color="lightgray")

# Show the plot
fig.show()

### Histogram of PNL Distribution
The histogram displays the distribution of PNL values across all trades. It helps in understanding the frequency and range of profit and loss outcomes.


In [ ]:
fig = px.histogram(executors_df, x='net_pnl_quote', title='PNL Distribution')
fig.show()
